## NSCP Section 407 One Way Slabs

In [3]:
import math

In [ ]:
def minthickness(clearspan, support, fy, wc):
    """
    Table 407.3.1.1 minimum thickness of one-way slabs

    clearspan: clear span length (mm)
    support: simply, one-end, both-end, or cantilever
    fy: yield strength (mpa)
    wc: unit weight of concrete (kg/m3)

    returns: minimum slab thickness (mm)
    """
    if support == 'simply':
        minthk = clearspan / 20
    elif support == 'one-end':
        minthk = clearspan / 24
    elif support == 'both-end':
        minthk = clearspan / 28
    elif support == 'cantilever':
        minthk = clearspan / 10
    else:
        raise ValueError("invalid support type; choose simply, one-end, both-end, or cantilever")

    # if fy equals 420 and wc is greater than 1840 then no adjustment
    if fy == 420 and wc > 1840:
        result = minthk
    # if fy is not 420 then adjust thickness
    elif fy != 420:
        result = minthk * (0.4 + fy / 700)
    # if using lightweight concrete (wc between 1440 and 1840) then apply multiplier
    elif 1440 <= wc <= 1840:
        multiplier = max(1.65 - 0.0003 * wc, 1.09)
        result = minthk * multiplier
    else:
        raise ValueError("invalid data; check fy and wc")

    return result

In [24]:
def minreinforcementarea(width, thickness):
    """
    Table 407.6.1.1 As,min for Non-Prestressed One-Way Slabs
    
    sectionwidth: width of the slab section (mm): typically 1000 for one-way design
    thickness: slab thickness (mm)"
    """
    areagross = width * thickness

    if fy < 420:
        asmin = 0.0020 * areagross
    else:
        asmin = max(0.0018 * 420 * areagross / fy, 0.0014 * areagross)

    result = asmin
        
    return result

In [ ]:
def minshearreinforcement(fc, width, fyt, vu, phi, vc):
    """
    NSCP 2015 407.6.3
    Minimum shear Reinforcement
    """

    phivc = phi * vc
    
    if vu > phivc:
        avmin = max (0.062 * math.sqrt (fc) * width / fyt, 0.35 * width / fyt)
        result = avmin
    else: 
        result = ValueError("Avmin is not required")

    return result

## To be continued...

In [ ]:
def minimumspacing (barsize, dagg):
    """425.2.1 For parallel non-prestressed reinforcement in a horizontal layer"""
    result = max (25, barsize, 4/3 * dagg) # clear spacing
    return result

    """425.2.2 For parallel non-prestressed reinforcement placed in two or more horizontal layers,
    reinforcement in the upper layers shall be placed directly above the reinforcement in the bottom layer
    with a clear spacing between layers of at least 25mm"""

In [ ]:
def maximumspacing ():

In [ ]:
def grossmomentofinertia(sectionwidth, thickness):
    """
    calculate gross moment of inertia for a rectangular section
    formula: inertia = (sectionwidth * thickness^3) / 12  # comment: thickness cubed times sectionwidth divided by 12
    sectionwidth: width of the section (mm)
    thickness: slab thickness (mm)
    returns: gross moment of inertia (mm4)  # comment: in mm4
    """
    return sectionwidth * math.pow(thickness, 3) / 12

def effectivedepth(thickness, cover, bardiameter):
    """
    calculate effective depth of the slab
    formula: effective depth = thickness - cover - (bardiameter / 2)  # comment: in mm
    thickness: slab thickness (mm)
    cover: concrete cover (mm)
    bardiameter: diameter of reinforcement bars (mm)
    returns: effective depth (mm)
    """
    return thickness - cover - (bardiameter / 2)

def nominalmomentcapacity(reinforcementarea, yieldstrength, effectivedepthvalue, concretecompressivestrength, sectionwidth):
    """
    calculate nominal moment capacity using rectangular stress block method
    compute a = (reinforcementarea * yieldstrength) / (0.85 * concretecompressivestrength * sectionwidth)
    then moment capacity = reinforcementarea * yieldstrength * (effectivedepthvalue - a / 2)
    reinforcementarea: tensile reinforcement area (mm2)
    yieldstrength: yield strength of reinforcement (mpa)
    effectivedepthvalue: effective depth (mm)
    concretecompressivestrength: concrete compressive strength (mpa)
    sectionwidth: width of the slab section (mm)
    returns: nominal moment capacity (n-mm)  # comment: in n-mm
    """
    avalue = (reinforcementarea * yieldstrength) / (0.85 * concretecompressivestrength * sectionwidth)
    return reinforcementarea * yieldstrength * (effectivedepthvalue - avalue / 2)

def slabdeflection(uniformload, clearspan, modulus, inertiavalue):
    """
    calculate immediate deflection of a simply supported one-way slab under uniform load
    formula: deflection = (5 * uniformload * clearspan^4) / (384 * modulus * inertiavalue)
    uniformload: uniform load intensity (n/mm)
    clearspan: clear span (mm)
    modulus: modulus of elasticity (n/mm2)
    inertiavalue: moment of inertia (mm4)
    returns: deflection (mm)
    """
    return (5 * uniformload * math.pow(clearspan, 4)) / (384 * modulus * inertiavalue)



minimum slab thickness: 300.00
minimum reinforcement area: 600.00
gross moment of inertia: 2250000000.00
effective depth: 252.00
design moment capacity: 55.81
short-term slab deflection: 0.32


In [ ]:
def main():
    # sample input values
    clearspan = 6000                   # clear span length (mm)
    supporttype = 'simply'             # support type: simply, one-end, both-end, cantilever
    sectionwidth = 1000                # section width (mm)
    concretecompressivestrength = 25   # concrete compressive strength (mpa)
    yieldstrength = 420                # yield strength (mpa)
    cover = 40                         # concrete cover (mm)
                                        """"
                                        Table 420.6.1.3.1 - Concrete Cover
                                        With Ground                  :  75mm
                                        Slabs with 40mm and 50mm bars:  40mm
                                        Slabs with 36mm and smaller  :  20mm
                                        """
    bardiameter = 16                   # bar diameter (mm)
    uniformload = 1                    # uniform load intensity (n/mm)

    modulus = 4700 * math.sqrt(concretecompressivestrength)  # modulus of elasticity (n/mm2)

    minimumthickness = minthickness(clearspan, supporttype)
    print("minimum slab thickness: {:.2f}".format(minimumthickness))

    slabthickness = minimumthickness  # assume slab thickness is at least the minimum required
    minreinforcementareavalue = minreinforcementarea(sectionwidth, slabthickness)
    print("minimum reinforcement area: {:.2f}".format(minreinforcementareavalue))

    grossinertia = grossmomentofinertia(sectionwidth, slabthickness)
    print("gross moment of inertia: {:.2f}".format(grossinertia))

    effectivedepthvalue = effectivedepth(slabthickness, cover, bardiameter)
    print("effective depth: {:.2f}".format(effectivedepthvalue))

    nominalmomentcap = nominalmomentcapacity(minreinforcementareavalue, yieldstrength, effectivedepthvalue, concretecompressivestrength, sectionwidth)
    strengthreductionfactor = 0.9
    designmomentcap = strengthreductionfactor * nominalmomentcap
    print("design moment capacity: {:.2f}".format(designmomentcap / 1e6))

    deflectionvalue = slabdeflection(uniformload, clearspan, modulus, grossinertia)
    print("short-term slab deflection: {:.2f}".format(deflectionvalue))

if __name__ == '__main__':
    main()